In [11]:
from aster_timestamps import aster_timestamps
import pandas as pd
import numpy as np

Find all the aster files, their filepaths, and timestamps:

In [12]:
aster_files = aster_timestamps('/storage/spestana/ASTER/AST_L1T/hdf')

Find the four ASTER images I want to try and make NDVI maps from:

In [29]:
my_timestamps = [pd.Timestamp('2017-05-23T18:51:10', tz='UTC'),
                 pd.Timestamp('2018-05-17T18:58:31', tz='UTC'),
                 pd.Timestamp('2019-04-27T18:51:22', tz='UTC'),
                 pd.Timestamp('2020-04-13T18:51:17', tz='UTC')]

my_filepaths = [aster_files.loc[aster_files.timestampUTC==this_timestamp].filepath.values[0] for this_timestamp in my_timestamps]

In [31]:
my_filepaths

['/storage/spestana/ASTER/AST_L1T/hdf/AST_L1T_00305232017185110_20170524170415_26127.hdf',
 '/storage/spestana/ASTER/AST_L1T/hdf/AST_L1T_00305172018185831_20180518114917_21406.hdf',
 '/storage/spestana/ASTER/AST_L1T/hdf/AST_L1T_00304272019185122_20190428120146_10912.hdf',
 '/storage/spestana/ASTER/AST_L1T/hdf/2020/New folder/both/AST_L1T_00304132020185117_20200414091306_12717.hdf']

Copy these files to a local data directory

In [41]:
#!cp "{my_filepaths[3]}" ./data/

For each of these files, now run the ASTERL1T_DN2REF.py utility to convert DN hdfs to reflectance geotiffs.

In [47]:
#!python ../aster-l1t/ASTERL1T_DN2REF.py ~/git/aster/data/

Processing File: AST_L1T_00305232017185110_20170524170415_26127.hdf (1 out of 4)
Processing File: AST_L1T_00305172018185831_20180518114917_21406.hdf (2 out of 4)
Processing File: AST_L1T_00304272019185122_20190428120146_10912.hdf (3 out of 4)
Processing File: AST_L1T_00304132020185117_20200414091306_12717.hdf (4 out of 4)


Now open the Red (band 2) and NIR (band 3) files in `~/git/aster/data/output/` to make NDVI maps!

In [134]:
#import xarray as xr
#import rioxarray

import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

In [135]:
def aster_ndvi(red_filepath, nir_filepath, ndvi_filepath):
    
    reader_red = rio.open(red_filepath)
    red = reader_red.read(1)

    reader_nir = rio.open(nir_filepath)
    nir = reader_nir.read(1)

    ndvi = np.zeros(reader_red.shape, dtype=rio.float32)
    ndvi = (nir.astype(float) - red.astype(float)) / (nir + red)

    kwargs = reader_red.meta
    kwargs.update(
        dtype=rio.float32,
        count=1,
        compress='lzw')

    with rio.open(ndvi_filepath, 'w', **kwargs) as dst:
        dst.write_band(1, ndvi.astype(rio.float32))

In [153]:
red_filepath = 'data/output/AST_L1T_00304132020185117_20200414091306_12717_ImageData2_reflectance.tif'
nir_filepath = 'data/output/AST_L1T_00304132020185117_20200414091306_12717_ImageData3N_reflectance.tif'
ndvi_filepath = 'data/AST_L1T_00304132020185117_20200414091306_12717_NDVI.tif'

In [154]:
aster_ndvi(red_filepath, nir_filepath, ndvi_filepath)

/home/spestana/opt/anaconda3/envs/goes-linux/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/home/spestana/opt/anaconda3/envs/goes-linux/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
AST_L1T_00305172018185831_20180518114917_21406_NDVI.tif